# <center>Data Science 2 - Assignment 1<center>
<center>Created by Zsófia Rebeka Katona<center> 

---

### 1.1. General information
You are required to submit two files to Moodle: an .ipynb file and the rendered .pdf file with
your solutions. Do not zip them together so I will be able to annotate the .pdf directly.
Please give short (2-3 sentences) interpretations / explanations to your answers, not only the
program code and outputs. Be concise and focused (less could be more ;)).
Grades will be distributed with the following rule: from the points you earn, you get 100% if you
submit until the due date (2024-03-22 20:00), 50% within 24 hours past due date, and 0% after


In [1]:
# Importing required libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### 1.2. Predict real estate value (20 points)
In this exercise you will predict property prices in New Taipei City, Taiwan using this dataset. (I
have uploaded the data to the repo for you with cleaned up variable names. You can find it in the
real_estate folder, here.) Let’s say you want to build a simple web app where potential buyers
and sellers could rate their homes, and the provided .csv contains the data you have collected.
Similarly to what we did in the class, let’s just work with a 20% subsample of the original data
first. Put aside 30% of that sample for the test set. (Hint: Extend the snippet below.)


In [2]:
# Setting the RandomState with Pseudo Random Number Generator
prng = np.random.RandomState(20240322)

# Importing the Taipei real estate dataset
real_estate_data = pd.read_csv("https://raw.githubusercontent.com/divenyijanos/ceu-ml/2023/data/real_estate/real_estate.csv")

# Getting a validation set, which is 20% subsample of the original data
real_estate_sample = real_estate_data.sample(frac=0.2)

In [3]:
# Checking the DataFrame
real_estate_data.head(5)

,id,transaction_date,house_age,distance_to_the_nearest_MRT_station,number_of_convenience_stores,latitude,longitude,house_price_of_unit_area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [4]:
# Checking the columns
print(real_estate_data.columns)

Index(['id', 'transaction_date', 'house_age',
       'distance_to_the_nearest_MRT_station', 'number_of_convenience_stores',
       'latitude', 'longitude', 'house_price_of_unit_area'],
      dtype='object')


I have considered choosing the following variables for the features: `house_age`, `distance_to_the_nearest_MRT_station` and `number_of_convenience_stores`.

In [5]:
# Defining the features: keep numeric features and putting them in the features variable
features = real_estate_sample[["house_age",
                               "distance_to_the_nearest_MRT_station",
                               "number_of_convenience_stores",
                               ]]

# Defining the target variable
outcome = real_estate_sample["house_price_of_unit_area"]

# Splitting the training and test samples
X_train, X_test, y_train, y_test = train_test_split(features, outcome, test_size=0.3, random_state=prng)

# Printing the size of the training and the test samples
print(f"Size of the training set: {X_train.shape}, size of the test set: {X_test.shape}")

Size of the training set: (58, 3), size of the test set: (25, 3)


### (2 points) Think about an appropriate loss function you can use to evaluate your predictive models. What is the risk (from a business perspective) that you would have to take by making a wrong prediction?

The primary goal in real estate is to maximize the profit for property investors, or to minimize the risks. Therefore, making the wrong decisions can lead to financial implications, such as losses. For instance, wrong pricing can lead to difficulties in selling it, resulting in the amortization of the property ( while it's not sold), selling it below market value or missing an investment opportunity (for example, if the real estate sector is booming at that certain time).
Consdiering the impact of wrong predictions, an appropriate loss model for real estate case could be the RMSE or the RMSLE. These loss function measure the average difference between the predicted the actual house prices, providing a clear understanding of the magnitude of prediction errors.

In [6]:
# Defining an alternative loss function
def calculateRMSLE(prediction, y_obs):
    return round(np.sqrt(
        np.mean(
                (
                    np.log(np.where(prediction < 0, 0, prediction) + 1) -
                    np.log(y_obs + 1)
                )**2
        )
    ), 4)

### (2 points) Build a simple benchmark model and evaluate its performance on the hold-out set (using your chosen loss function).

In [7]:
# Estimating a benchmark by choosing y_train as our benchmark and taking its average
benchmark = np.mean(y_train)
# Calculating the RMSLE based on the benchmark, the y_train and the y_test data
benchmark_result = ["Benchmark", calculateRMSLE(benchmark, y_train), calculateRMSLE(benchmark, y_test)]

In [38]:
# Defining the columns of our results dataframe
result_columns = ["Model", "Train", "Test"]
# We are putting the benchmark results into a DataFrame and using the predefined variables
results_df = pd.DataFrame([benchmark_result], columns=result_columns)
print(f"We can conclude that the train RMSLE is slightly higher ({calculateRMSLE(benchmark, y_train)}) compared to the test RMSLE ({calculateRMSLE(benchmark, y_test)}), which suggests \n that the test set is performing better with the simple benchmark model. This indicates that the model performs better on \n unseen data, and its performance generalizes well to new data.")
print(" "*10)
results_df

We can conclude that the train RMSLE is slightly higher (0.373) compared to the test RMSLE (0.3349), which suggests 
 that the test set is performing better with the simple benchmark model. This indicates that the model performs better on 
 unseen data, and its performance generalizes well to new data.
          


,Model,Train,Test
0,Benchmark,0.373,0.3349


### (2 points) Build a simple linear regression model using a chosen feature and evaluate its performance. Would you launch your evaluator web app using this model?

As I have chosen only 3 features in total: `house_age`, `distance_to_the_nearest_MRT_station` and `number_of_convenience_stores`, I have considered `house_age` as an important feature which I could use for the simple Linear Regression. Therefore, I could use all the features for the multivariate regression later on.

In [39]:
from sklearn.linear_model import LinearRegression

# Initialize the linear regression model
lin_reg = LinearRegression().fit(X_train[["house_age"]], y_train)

# Predictions on training and testing sets
train_predictions = lin_reg.predict(X_train[["house_age"]])
test_predictions = lin_reg.predict(X_test[["house_age"]])

# Calculating RMSLE for the training and testing sets
model_train_rmsle = calculateRMSLE(train_predictions, y_train)
model_test_rmsle = calculateRMSLE(test_predictions, y_test)

# Preparing the model's results
model_result = pd.DataFrame([["Linear Regression", model_train_rmsle, model_test_rmsle]],
                            columns=["Model", "Train", "Test"])

# Appending model_result to the existing results_df DataFrame
results_df = pd.concat([results_df, model_result], ignore_index=True)

# Displaying the updated results
print(f"The linear model is performing slighlty better comparing to the simple benchmark. ({model_train_rmsle}) for the training set and ({model_test_rmsle}) \n for the test set. The difference only includes a few decimals. The training set is still performing weaker than the test \n set, suggesting that this model is also permorming better on unseen data.")
print(" "*10)
results_df

The linear model is performing slighlty better comparing to the simple benchmark. (0.3503) for the training set and (0.3124) 
 for the test set. The difference only includes a few decimals. The training set is still performing weaker than the test 
 set, suggesting that this model is also permorming better on unseen data.
          


,Model,Train,Test
0,Benchmark,0.3730,0.3349
1,Linear Regression,0.3503,0.3124


### (2 points) Build a multivariate linear model with all the meaningful variables available. Did it improve the predictive power?

In [40]:
# Setting the first group of features
features = ["house_age", "distance_to_the_nearest_MRT_station", "number_of_convenience_stores"]

# Initialize the linear regression model
multi_lin_reg = LinearRegression()
multi_lin_reg.fit(X_train[features], y_train)

# Predictions on training and testing sets
train_predictions_multi = multi_lin_reg.predict(X_train[features])
test_predictions_multi = multi_lin_reg.predict(X_test[features])

# Calculating the errors
multi_model_rmsle_train = calculateRMSLE(train_predictions_multi, y_train)
multi_model_rmsle_test = calculateRMSLE(test_predictions_multi, y_test)

# Preparing the model's results
multi_model_result = pd.DataFrame([["Multivariate Regression", multi_model_rmsle_train, multi_model_rmsle_test]],
                            columns=["Model", "Train", "Test"])

# Appending model_result to the existing results_df DataFrame
results_df = pd.concat([results_df, multi_model_result], ignore_index=True)

# Displaying the updated results
print(f"The multivariate regression improved our RMSLE scores more than the simple linear model, suggesting that including additional \n features in the model has helped capture more of the variability in the target variable. This resulted in more accurate \n predictions. Based on that, we can conclude that the selected features have meaningful relationships with the target variable \n and contribute positively to the predictive performance of the model.")
print(" "*10)
results_df

The multivariate regression improved our RMSLE scores more than the simple linear model, suggesting that including additional 
 features in the model has helped capture more of the variability in the target variable. This resulted in more accurate 
 predictions. Based on that, we can conclude that the selected features have meaningful relationships with the target variable 
 and contribute positively to the predictive performance of the model.
          


,Model,Train,Test
0,Benchmark,0.3730,0.3349
1,Linear Regression,0.3503,0.3124
2,Multivariate Regression,0.2129,0.2068


### (6 points) Try to make your model (even) better. Document your process and its success while taking two approaches:
    1. Feature engineering - e.g. including squares and interactions or making sense of latitude&longitude by calculating the distance from the city center, etc.
    2. Training more flexible models - e.g. random forest or gradient boosting

In [11]:
# Creating the squared variables
real_estate_sample["house_age_sq"] = real_estate_sample["house_age"] ** 2
real_estate_sample["distance_to_the_nearest_MRT_station_sq"] = real_estate_sample["distance_to_the_nearest_MRT_station"] ** 2
real_estate_sample["number_of_convenience_stores_sq"] = real_estate_sample["number_of_convenience_stores"] ** 2

# Creating the interactions
real_estate_sample["house_age_distance_interaction"] = real_estate_sample["house_age"] * real_estate_sample["distance_to_the_nearest_MRT_station"]
real_estate_sample["house_age_stores_interaction"] = real_estate_sample["house_age"] * real_estate_sample["number_of_convenience_stores"]
real_estate_sample["distance_stores_interaction"] = real_estate_sample["distance_to_the_nearest_MRT_station"] * real_estate_sample["number_of_convenience_stores"]

# Creating the interactions between the squared variables
real_estate_sample["house_age_distance_interaction_sq"] = real_estate_sample["house_age"] * real_estate_sample["distance_to_the_nearest_MRT_station"] * real_estate_sample["house_age_sq"] * real_estate_sample["distance_to_the_nearest_MRT_station_sq"]
real_estate_sample["house_age_stores_interaction_sq"] = real_estate_sample["house_age"] * real_estate_sample["number_of_convenience_stores"] * real_estate_sample["house_age_sq"] * real_estate_sample["number_of_convenience_stores_sq"]
real_estate_sample["distance_stores_interaction_sq"] = real_estate_sample["distance_to_the_nearest_MRT_station"] * real_estate_sample["number_of_convenience_stores"] * real_estate_sample["distance_to_the_nearest_MRT_station_sq"] * real_estate_sample["number_of_convenience_stores_sq"]

real_estate_sample.head(5)

,id,transaction_date,house_age,distance_to_the_nearest_MRT_station,number_of_convenience_stores,latitude,longitude,house_price_of_unit_area,house_age_sq,distance_to_the_nearest_MRT_station_sq,number_of_convenience_stores_sq,house_age_distance_interaction,house_age_stores_interaction,distance_stores_interaction,house_age_distance_interaction_sq,house_age_stores_interaction_sq,distance_stores_interaction_sq
176,177,2012.833,13.9,4573.7790,0,24.94867,121.49507,19.2,193.21,2.091945e+07,0,63575.52810,0.0,0.0000,2.569626e+14,0.000,0.000000e+00
9,10,2013.417,17.9,1783.1800,3,24.96731,121.51486,22.1,320.41,3.179731e+06,9,31918.92200,53.7,5349.5400,3.251956e+13,154854.153,1.530909e+11
241,242,2013.500,13.7,250.6310,7,24.96606,121.54297,41.4,187.69,6.281590e+04,49,3433.64470,95.9,1754.4170,4.048238e+10,881974.079,5.400059e+09
332,333,2013.167,39.8,617.7134,2,24.97577,121.53475,39.6,1584.04,3.815698e+05,4,24584.99332,79.6,1235.4268,1.485971e+13,504358.336,1.885606e+09
369,370,2012.667,20.2,2185.1280,3,24.96322,121.51237,22.8,408.04,4.774784e+06,9,44139.58560,60.6,6555.3840,8.599729e+13,222545.016,2.817049e+11


In [12]:
# Making sense of the latitude and longitude values
import numpy as np

# Defining the coordinates of the reference point (e.g., city center)
city_center = (25.0330, 121.5654)

# Calculating the distance from each data point to the reference point using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Converting latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371 
    return c * r

# Calculating the distance from each data point to the city center
real_estate_sample['distance_to_city_center'] = haversine(real_estate_sample['latitude'], real_estate_sample['longitude'], city_center[0], city_center[1])

In [13]:
# Checking the distance to the city center variable
real_estate_sample.head(5)

,id,transaction_date,house_age,distance_to_the_nearest_MRT_station,number_of_convenience_stores,latitude,longitude,house_price_of_unit_area,house_age_sq,distance_to_the_nearest_MRT_station_sq,number_of_convenience_stores_sq,house_age_distance_interaction,house_age_stores_interaction,distance_stores_interaction,house_age_distance_interaction_sq,house_age_stores_interaction_sq,distance_stores_interaction_sq,distance_to_city_center
176,177,2012.833,13.9,4573.7790,0,24.94867,121.49507,19.2,193.21,2.091945e+07,0,63575.52810,0.0,0.0000,2.569626e+14,0.000,0.000000e+00,11.754635
9,10,2013.417,17.9,1783.1800,3,24.96731,121.51486,22.1,320.41,3.179731e+06,9,31918.92200,53.7,5349.5400,3.251956e+13,154854.153,1.530909e+11,8.904797
241,242,2013.500,13.7,250.6310,7,24.96606,121.54297,41.4,187.69,6.281590e+04,49,3433.64470,95.9,1754.4170,4.048238e+10,881974.079,5.400059e+09,7.779048
332,333,2013.167,39.8,617.7134,2,24.97577,121.53475,39.6,1584.04,3.815698e+05,4,24584.99332,79.6,1235.4268,1.485971e+13,504358.336,1.885606e+09,7.073652
369,370,2012.667,20.2,2185.1280,3,24.96322,121.51237,22.8,408.04,4.774784e+06,9,44139.58560,60.6,6555.3840,8.599729e+13,222545.016,2.817049e+11,9.421582


In [14]:
# Setting the second set of features with feature engineering
features_fe = real_estate_sample[[
    "house_age_sq",
    "distance_to_the_nearest_MRT_station_sq",
    "number_of_convenience_stores_sq",
    "house_age_distance_interaction",
    "house_age_stores_interaction",
    "distance_stores_interaction",
    "house_age_distance_interaction_sq",
    "house_age_stores_interaction_sq",
    "distance_stores_interaction_sq",
    "distance_to_city_center"
]]

# Splitting the data again with the feature engineered variables
X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(features_fe, outcome, test_size=0.3, random_state=prng)

# Printing the size of the training and the test samples
print(f"Size of the training set: {X_train_fe.shape}, size of the test set: {X_test_fe.shape}")

Size of the training set: (58, 10), size of the test set: (25, 10)


In [41]:
# Initialize the linear regression model
multi_lin_reg_fe = LinearRegression()
multi_lin_reg_fe.fit(X_train_fe, y_train_fe)

# Predictions on training and testing sets
train_predictions_multi_fe = multi_lin_reg_fe.predict(X_train_fe)
test_predictions_multi_fe = multi_lin_reg_fe.predict(X_test_fe)

# Calculating the errors
multi_model_rmsle_train_fe = calculateRMSLE(train_predictions_multi_fe, y_train_fe)
multi_model_rmsle_test_fe = calculateRMSLE(test_predictions_multi_fe, y_test_fe)

# Preparing the model's results
multi_model_result_fe = pd.DataFrame([["Multivariate Regression with FE", multi_model_rmsle_train_fe, multi_model_rmsle_test_fe]],
                            columns=["Model", "Train", "Test"])

# Appending model_result to the existing results_df DataFrame
results_df = pd.concat([results_df, multi_model_result_fe], ignore_index=True)

# Displaying the updated results
print(f"We can see that the feature engineered multivariate regression gave us surprising results. We can see that the training RMLSE \n ({multi_model_rmsle_train_fe}) has improved compared to the original multivariate regression. However, the test RMSLE ({multi_model_rmsle_test_fe}) increased \n significantly. This indicates that the feature engineered model may have overfitted to the training data, it failed to \n generalize well to unseen data.")
print(" "*10)
results_df

We can see that the feature engineered multivariate regression gave us surprising results. We can see that the training RMLSE 
 (0.1777) has improved compared to the original multivariate regression. However, the test RMSLE (0.1904) increased 
 significantly. This indicates that the feature engineered model may have overfitted to the training data, it failed to 
 generalize well to unseen data.
          


,Model,Train,Test
0,Benchmark,0.3730,0.3349
1,Linear Regression,0.3503,0.3124
2,Multivariate Regression,0.2129,0.2068
3,Multivariate Regression with FE,0.1777,0.1904


### RandomForest with feature engineering
---

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

steps = [
    ("random_forest", RandomForestRegressor())
]
pipe_rf = Pipeline(steps)


pipe_rf.fit(X_train_fe, y_train_fe)

train_error_rf = calculateRMSLE(pipe_rf.predict(X_train_fe), y_train_fe)
test_error_rf = calculateRMSLE(pipe_rf.predict(X_test_fe), y_test_fe)

# Preparing the model's results
result_rf = pd.DataFrame([["Random Forest with FE", train_error_rf, test_error_rf]],
                            columns=["Model", "Train", "Test"])

# Appending model_result to the existing results_df DataFrame
results_df = pd.concat([results_df, result_rf], ignore_index=True)

print(f"We can see that RandomForest improves our model. It performs well on the test set ({test_error_rf}), but even better on the training \n set ({train_error_rf}). This suggests that the RandomForest model may have captured complex patterns present in the training data more \n effectively than the previous models, potentially leading to overfitting.")
print(" "*10)
results_df

We can see that RandomForest improves our model. It performs well on the test set (0.1913), but even better on the training 
 set (0.0484). This suggests that the RandomForest model may have captured complex patterns present in the training data more 
 effectively than the previous models, potentially leading to overfitting.
          


,Model,Train,Test
0,Benchmark,0.3730,0.3349
1,Linear Regression,0.3503,0.3124
2,Multivariate Regression,0.2129,0.2068
3,Multivariate Regression with FE,0.1777,0.1904
4,Random Forest with FE,0.0484,0.1913


### Gradient Boosting with feature engineering
---

In [43]:
from sklearn import tree
# Setting the pipeline
steps = [
    ("deep_tree", tree.DecisionTreeRegressor(max_depth = 10, random_state = prng))
]
pipe_tree_deep = Pipeline(steps)

# Illustration
pipe_tree_deep.fit(X_train_fe, y_train_fe)

train_error_gradient = calculateRMSLE(pipe_tree_deep.predict(X_train_fe), y_train_fe)
test_error_gradient = calculateRMSLE(pipe_tree_deep.predict(X_test_fe), y_test_fe)

# Preparing the model's results
result_gradient = pd.DataFrame([["Gradient Boosting with FE", train_error_gradient, test_error_gradient]],
                            columns=["Model", "Train", "Test"])

# Appending model_result to the existing results_df DataFrame
results_df = pd.concat([results_df, result_gradient], ignore_index=True)

print(f"From the RMSLE scores, it can be concluded that the feature engineered Gradient Boosting model is experiencing overfitting. \n Although the training set shows a relatively low RMSLE ({train_error_gradient}), the performance on the test set ({test_error_gradient}) is worse to that \n of previous models. This indicates that the model is learning the training data too closely, leading to a lack of \n generalization ability when applied to unseen data.")
print(" "*10)
results_df

From the RMSLE scores, it can be concluded that the feature engineered Gradient Boosting model is experiencing overfitting. 
 Although the training set shows a relatively low RMSLE (0.0141), the performance on the test set (0.2444) is worse to that 
 of previous models. This indicates that the model is learning the training data too closely, leading to a lack of 
 generalization ability when applied to unseen data.
          


,Model,Train,Test
0,Benchmark,0.3730,0.3349
1,Linear Regression,0.3503,0.3124
2,Multivariate Regression,0.2129,0.2068
3,Multivariate Regression with FE,0.1777,0.1904
4,Random Forest with FE,0.0484,0.1913
5,Gradient Boosting with FE,0.0141,0.2444


### (2 points) Would you launch your web app now? What options you might have to further improve the prediction performance?

I could introduce 

### (4 points) Rerun three of your previous models (including both flexible and less flexible ones) on the full train set. Ensure that your test result remains comparable by keeping that dataset intact. (Hint: extend the code snippet below.) Did it improve the predictive power of your models? Where do you observe the biggest improvement? Would you launch your web app now?